In [7]:
# Load packages
import json
import numpy as np
import pandas as pd
import pickle
import requests

from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players, teams

from nba import NbaScraper, ShotCharts

In [8]:
# Variables
YEARS = ['2018-19', '2019-20', '2020-21', '2021-22', '2022-23']

In [9]:
# Load players
with open('./data/players_career.pickle', 'rb') as f:
    players_career = pickle.load(f)

In [10]:
# Number of players
print(f"Number of players: {len(players_career)}")

Number of players: 531


In [11]:
# Create a dataset containing the players with the year
# in YEARS in NBA
players = list()
for pp in players_career:
    temp = pp['career']
    temp = temp[temp['SEASON_ID'].isin(YEARS)]
    if len(temp) > 4:
        players.append({
            'PLAYER_ID': pp['id'],
            'full_name': pp['full_name'],
            'career': temp[['PLAYER_ID', 'SEASON_ID', 'TEAM_ID']]
        })
    else:
        continue

In [12]:
players_season_long = pd.concat([pp['career'] for pp in players])
players_season_long = players_season_long.query('TEAM_ID > 0')

In [13]:
players_season_long

,PLAYER_ID,SEASON_ID,TEAM_ID
5,203500,2018-19,1610612760
6,203500,2019-20,1610612760
7,203500,2020-21,1610612740
8,203500,2021-22,1610612763
9,203500,2022-23,1610612763
...,...,...,...
3,1627826,2018-19,1610612746
5,1627826,2019-20,1610612746
6,1627826,2020-21,1610612746
7,1627826,2021-22,1610612746


In [117]:
df = pd.DataFrame()
for idx, row in players_season_long.iterrows():
    shot_data = shotchartdetail.ShotChartDetail(
        team_id=row['TEAM_ID'],
        player_id=row['PLAYER_ID'],
        context_measure_simple='FGA',
        season_nullable=row['SEASON_ID']
    )
    df = pd.concat([df, shot_data.get_data_frames()[0]])

In [119]:
# Save shots
with open('./data/player_shots.pickle', 'wb') as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)